# Использование предобученных моделей

Задание: собрать датасет и использовать перенос обучения для решения задачи классификации.

## Порядок выполнения

1. Скачать изображения для создания датасета. Как структурировать папки подсмотрите в работе по ConvNet в датасете с Симпсонами;
1. Подготовить transforms, DataSet и DataLoader;
1. Выбрать одну из моделей в библиотеке timm;
1. Использовать на этой модели прием выделения признаков;
1. Использовать на этой модели прием дообучения (fine-tune);
1. Оценить результаты лучшей модели на тестовой выборке.

## Источники

1. [Туториал от Pytorch](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)
1. [GitHub pytorch-image-models](https://github.com/huggingface/pytorch-image-models)
1. [Извлечение признаков](https://huggingface.co/docs/timm/feature_extraction)
1. [Which image models are best?](https://www.kaggle.com/code/jhoward/which-image-models-are-best)
1. [Pytorch Image Models (timm)](https://timm.fast.ai/)
1. [huggingface docs timm](https://huggingface.co/docs/hub/timm)

## Создание датасета

В этом задании вам надо собрать собственный датасет из изображений. В нем должно быть минимум 30 изображений для каждого класса. Количество классов не менее 2. Тематику датасета вы выбираете самостоятельно.

Далее в этом разделе приведен пример кода, который помогает скачать изображения по запросу на гугл диск. Вы можете решить эту задачу другими удобными для вас способами, в том числе и вручную.

Если вы работаете с локальной средой, то код из примера придется модифицировать.

In [ ]:
# Установка пакета для работы с API поисковика DuckDuckGo

!pip install -U duckduckgo_search

In [ ]:
from duckduckgo_search import DDGS  # импотр модуля

with DDGS() as ddgs:
    ddgs_images_gen = ddgs.images(
      'owl',  # пример получения изображений по запросу 'owl'
      region="wt-wt",
      size="Medium",
      type_image="photo",
      max_results=20,  # максимальное количество изображений в ответе
    )
    with open('owl.txt', 'w') as f:  # пишем в файл полученные ссылки на изображения для скачивания
      for r in ddgs_images_gen:
        f.write(f"{r['image']}\n")

In [ ]:
# монтируем гугл диск к среде, чтобы можно было записывать и считывать изображения
# в постоянное хранилище на гугл диске. Колаб попросит предоставить доступ.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# пример создания папки dataset в корне вашего гугл диска
!mkdir "/content/drive/My Drive/dataset"

In [ ]:
# можно посмотреть содержимое файла со ссылками
!cat owl.txt

In [ ]:
# утилита wget построчно читает файл owl.txt и скачивает по URL файлы в папку,
# указанную после флага -P. --random-wait добавляет случайные интервалы между запросами,
# чтобы снизить вероятность блокировки

!wget -i owl.txt --random-wait -P "/content/drive/My Drive/dataset/owl"

### Ответы по датасету (макс. 20 баллов)

В результате работы по этому разделу у вас должен получиться датасет. Проверьте что все скачанные изображения открываются и удалите поврежденные файлы. Изображения стоит разделить в папках на train и test и примерном соотношении 80 и 20%.

Доступ к вашему датасету потребуется открыть. Если вы скачивали изображения локально, то их надо будет выгрузить на гугл или яндекс диск.

**Ссылка на ваш датасет - https://**

**Описание вашего датасета:**

* Общее описание решаемой задачи:
* Количество классов:
* Имена классов:
* Количество изображений в каждом классе в обучающей и тестовой выборках

---

Ваш ответ:

## Импортирование модулей

In [ ]:
# по необходимости добавляйте свои модули

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Использование GPU по желанию

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Работаем на GPU')
else:
  device = torch.device('cpu')
  print('Работаем на CPU')

# Не забывайте про .to(device)

## Выбор модели из timm

В источниках к заданию, вы найдете список моделей, который выложен на github и [сравнение части моделей из timm](https://www.kaggle.com/code/jhoward/which-image-models-are-best). Вам необходимо остановить свой выбор на одной из них. В списке моделей на github есть ссылки на статьи о них, где можно найти информацию на каком датасете они были обучены.

Если вы работаете с фотографиями выбор можно остановить на одной из:
1. MobileNet,
1. VGG,
1. ResNet,
1. Xception.

По желанию можете попробовать несколько и сравнить.


In [ ]:
# сначала требуется установить сам модуль timm
!pip install timm

In [ ]:
import timm

In [ ]:
# вывод списка моделей содержащих *resnet* и предобученных
# timm.list_models("*resnet*", pretrained=True)

In [ ]:
# в качестве первого аргумента укажите имя выбранной модели
# и не забудьте указать, предобученный вариант (pretrained)
pretrained_model = timm.create_model()

In [ ]:
# Вывод архитектуры модели
print(pretrained_model)

В описании архитектуры модели в выводе предыдущей ячейки, найдите два новых модуля (Conv2d, ReLU вы уже знаете) и добавьте их описание. (макс. 15 баллов)

**Ваш ответ:**

1. Модуль ___ - делает ...
1. Модуль ___ - делает ...

**Почему решили выбрать именно эту модель?** (макс. 5 баллов)

Ваш ответ:

## Создание Dataset и DataLoader

По аналогии с прошлыми заданиями нам требуется создать transforms, которые передаются в создаваемый Dataset и из датасета вы создаете DataLoaders.

Данных у нас немного, поэтому мы не будем выделять валидационную часть.

При создании transforms помните, что модель ожидает на вход тензор определенной размерности. В [описание моделей](https://paperswithcode.com/lib/timm), вы можете найти датасет, на котором обучалась модель и есть описание размера изображений.

Для работы вам пригодится:

- v2.ToImage()
- v2.Resize() или v2.RandomResizedCrop() - размер изображения после кадрирования, должен быть равен размеру ожидаемому на ходе предобученной модели.
- v2.RandomRotation()
- v2.RandomHorizontalFlip()
- v2.ToDtype()

[Описания в документации](https://pytorch.org/vision/stable/transforms.html#v2-api-reference-recommended)

In [ ]:
# transforms = v2.Compose([]) помните что для обучения и теста нужен разный набор преобразований

# train_dataset = ImageFolder(root="./data/train", transform=transforms)
# test_dataset = ImageFolder(root="./data/train", transform=transforms)

#
# Визуализируйте образцы чтобы убедиться, что train_dataset создан корректно
# Например используя, make_grid
# inputs, classes = next(iter(dataloader))
# out = torchvision.utils.make_grid(inputs)
# imshow(out, title=[class_names[x] for x in classes])

In [ ]:
#
# Ваш код
#

# train_loader = DataLoader()
# test_loader = DataLoader()

**Почему выбрали именно такие преобразования (transforms) для данных?** (макс. 10 баллов)

Ваш ответ:

## Прием извлечения признаков

Этот прием может использоваться как сам по себе, так и быть предварительным этапом для дообучения.

Он заключается в том, что мы заменяем полносвязную часть модели (head/голову) на свою с учетом размерностей выходных данных из сверточной части и количеством классов в текущей задаче. Перед обучением требуется "заморозить" параметры сверточных слоев.

**Почему требуется "заморозка" параметров?** (макс. 10 баллов)

Ваш ответ:



У модели вы можете использовать метод .parameters(), он возвращает итерируемый объект с параметрами вашей модели. Вы можете их перебрать и отключить необходимость расчета градиентов.

In [ ]:
# "Замораживаем" веса
# for param in ___:
    #param...

# Заменяем "голову"
# .fc для вашей модели может иметь другое имя
# В nn.Sequential добавьте 1-2 скрытых слоя (nn.Linear, nn.ReLU)
pretrained_model.fc = nn.Sequential()

In [ ]:
loss_fn =
optimizer =

В этой работе также рассмотрим применение планировщика для изменения скорости обучения. Ранее у вас скорость обучения была константой, теперь же в процессе обучения каждые n эпох будем ее снижать.



In [ ]:
from torch.optim import lr_scheduler

scheduler = lr_scheduler.StepLR()

### Обучение и тестирование

Несколько эпох обучите модель в таком состоянии. Для обучения используйте уже знакомый вам цикл с эпохами и перебором dataloader, но к нему в цикл эпох требуется добавить шаг планировщика scheduler.step().

Не забывайте переключать режимы моделей (pretrained_model.train(), pretrained_model.eval()), так как теперь в них может быть пакетная нормализация и используйте контекст torch.no_grad() при проверке модели.

In [ ]:
for epoch in range(num_epochs):
  for images, labels in loader:

  #
  # Ваш код
  #

  scheduler.step()

## Прием дообучения

Чтобы дообучить модель, требуется разморозить параметры модели. Для упрощения можете разморозить все параметры модели, но более правильно будет разморозить параметры 1-2 последних слоев и оптимизировать их, и наиболее сложный вариант разморозить больше 2 слоев, но использовать сниженные скорости обучения для более ранних слоев модели - [TORCH.OPTIM](https://pytorch.org/docs/stable/optim.html#per-parameter-options).

In [ ]:
# Первый вариант
# for param in ___:
    #param...

# Второй вариант
# model.blocks[-n:].requires_grad_(True)

# Третий вариант
# model.blocks[-n:].requires_grad_(True)
# optim.SGD([
#                 {'params': model.base.parameters()},
#                 {'params': model.classifier.parameters(), 'lr': 1e-3}
#             ], lr=1e-2, momentum=0.9)

In [ ]:
# Новый набор объектов

loss_fn =
optimizer =
scheduler =

In [ ]:
# Новый цикл обучения

for epoch in range(num_epochs):
  for images, labels in loader:

  #
  # Ваш код
  #

  scheduler.step()

Скачайте одно новое изображение, которое можно отнести к одному из ваших классов, и классифицируйте его с помощью полученной модели.

## Финал

Осталось попробовать модель на случайном изображении и ответить на вопросы.

In [ ]:
# import PIL
# img_file = "./robot_image.jpg"
# img = PIL.Image.open(img_file)
# img = transforms(img)
# img = img.unsqueeze(0)
# model.eval()

#
# Ваш код
#

**Правильно ли модель классифицировала ваше изображение?** (макс. 10 баллов)

Ваш ответ:

**Какая итоговая точность работы вашей модели на тестовой выборке?** (макс. 10 баллов)

Ваш ответ:

**Чем отличается прием извлечения признаков от дообучения?** (макс. 20 баллов)

Ваш ответ: